In [1]:
import os
import sys
import glob
import pandas as pd
import numpy as np
import xarray as xr
import datetime
import matplotlib.pyplot as plt
from itertools import groupby

In [2]:
fname = "data/BART_met.nc"

vars_to_keep = ['Tair','Qair','Rainf','Wind','PSurf','LWdown','SWdown','CO2air']
ds = xr.open_dataset(fname, decode_times=False)
freq = "0.5H"
units, reference_date = ds.time.attrs['units'].split('since')
df = ds[vars_to_keep].squeeze(dim=["x","y"], drop=True).to_dataframe()
start = reference_date.strip().split(" ")[0].replace("-","/")
df['dates'] = pd.date_range(start=start, periods=len(df), freq=freq)
df = df.set_index('dates')


In [3]:
for var in vars_to_keep:
    print("Checking NaN locations for variable ", var)
    for k,g in groupby(df[var].isnull().values):
        print((k, sum(1 for i in g))) 

Checking NaN locations for variable  Tair
(False, 245472)
Checking NaN locations for variable  Qair
(False, 245472)
Checking NaN locations for variable  Rainf
(False, 245472)
Checking NaN locations for variable  Wind
(False, 245472)
Checking NaN locations for variable  PSurf
(False, 245472)
Checking NaN locations for variable  LWdown
(False, 245472)
Checking NaN locations for variable  SWdown
(False, 245472)
Checking NaN locations for variable  CO2air
(False, 245472)


In [4]:
print(df[vars_to_keep[1]].isnull().values)

[False False False ... False False False]


In [14]:
swdown = df['SWdown'].values
print(swdown)

[155.48452684 155.48452684 155.48452684 ...   0.           0.
   0.        ]


In [6]:
print(np.sort(swdown))

[   0.            0.            0.         ... 1411.6        1429.30038462
 1467.2       ]


In [12]:
print(np.max(swdown))

1467.2
